### 3. <u>Routing</u>:
#### --> What is Routing in LangGraph?

Routing in LangGraph refers to the ability to conditionally determine which node to execute next based on the current state or the output of a node. This is typiacally implemented using:

* *<u>add_conditional_edges</u>*: A method that maps a node's output (or a condition function's result) to different possible next nodes.

* *<u>State*</u>: The workflow's state can store variables that influence routing decisions.

* *<u>Condition Functions*</u>: Functions that evaluate the state are node output to decide the next step.

### Key Concepts:

* *<u>Dynamic Flow*</u>: Unlike a linear sequence, routing lets the graph adapt to intermediate results.

* *<u>Condition Logic*</u>: You define rules (Example: "if this, go here; if that, go there.").

* *<u>Flexibility*</u>: Combines well with paralyzation or sequential chains for complex workflows.

In [3]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model='gemma3:270m')
response = llm.invoke("Hello")
response

AIMessage(content='Hello! How can I help you today?\n', additional_kwargs={}, response_metadata={'model': 'gemma3:270m', 'created_at': '2025-10-01T03:00:02.1506078Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1756661100, 'load_duration': 1411210600, 'prompt_eval_count': 10, 'prompt_eval_duration': 143715200, 'eval_count': 11, 'eval_duration': 194177100, 'model_name': 'gemma3:270m'}, id='run--73368bcf-c06f-44ac-80a5-c39a0c047690-0', usage_metadata={'input_tokens': 10, 'output_tokens': 11, 'total_tokens': 21})

**We implement Routing with the help of `pydantic` and `with_sturctured_output()` attribute/function of llm**

In [34]:
from typing_extensions import Literal,TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage,SystemMessage

# Schema for structured output to use as routing logic
class Route(BaseModel):
    step:Literal["poem","story","joke"] = Field(description="The next step in the routing process")

# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)

# state
class State(TypedDict):
    input:str
    decision:str
    output:str

# nodes
def llm_call_1(state:State):
    '''writes a story'''
    result = llm.invoke(state['input'])
    return {'output':result.content}

def llm_call_2(state:State):
    '''writes a joke'''
    print("llm call 2 is invoked")
    result = llm.invoke(state['input'])
    return {'output':result.content}

def llm_call_3(state:State):
    '''writes a poem'''
    print("llm call 3 is invoked")
    result = llm.invoke(state['input'])
    return {'output':result.content}

In [23]:
def llm_call_router(state:State):
    """Route the input to the appropriate node"""

    decision = router.invoke(
        [
            SystemMessage("Route the input to story, joke or poem based on the user's request"),
            HumanMessage(state['input'])
        ]
    )
    return {'decision':decision.step}

In [24]:
# Condition Function
def route_decision(state:State):
    if state['decision'] == 'story':
        return 'llm_call_1'
    elif state['decision'] == 'joke':
        return 'llm_call_2'
    elif state['decision'] == 'poem':
        return 'llm_call_3'

In [26]:
# Build the workflow
from langgraph.graph import StateGraph,START, END

builder = StateGraph(State)

builder.add_node("llm_call_1",llm_call_1)

In [27]:
def llm_call_router(state:State):
    """Route the input to the appropriate node"""

    decision = router.invoke(
        [
            SystemMessage("Route the input to story, joke or poem based on the user's request"),
            HumanMessage(state['input'])
        ]
    )
    return {'decision':decision.step}

In [28]:
# Condition Function
def route_decision(state:State):
    if state['decision'] == 'story':
        return 'llm_call_1'
    elif state['decision'] == 'joke':
        return 'llm_call_2'
    elif state['decision'] == 'poem':
        return 'llm_call_3'

In [29]:
# Build the workflow
from langgraph.graph import StateGraph,START, END

builder = StateGraph(State)

builder.add_node("llm_call_1",llm_call_1)

In [30]:
def llm_call_router(state:State):
    """Route the input to the appropriate node"""

    decision = router.invoke(
        [
            SystemMessage("Route the input to story, joke or poem based on the user's request"),
            HumanMessage(state['input'])
        ]
    )
    return {'decision':decision.step}

In [31]:
# Condition Function
def route_decision(state:State):
    if state['decision'] == 'story':
        return 'llm_call_1'
    elif state['decision'] == 'joke':
        return 'llm_call_2'
    elif state['decision'] == 'poem':
        return 'llm_call_3'

In [32]:
# Build the workflow
from langgraph.graph import StateGraph,START, END

builder = StateGraph(State)

builder.add_node("llm_call_1",llm_call_1)
builder.add_node("llm_call_2",llm_call_2)
builder.add_node("llm_call_3",llm_call_3)
builder.add_node("llm_call_router",llm_call_router)

builder.add_edge(START,"llm_call_router")
builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {
        'llm_call_1':'llm_call_1',
        'llm_call_2':'llm_call_2',
        'llm_call_3':'llm_call_3',
    },
)
builder.add_edge("llm_call_1",END)
builder.add_edge("llm_call_2",END)
builder.add_edge("llm_call_3",END)

builder_graph = builder.compile()

In [33]:
response = builder_graph.invoke({'input':'Write me a joke about agentic ai system'})

print(response['output'])

Why did the agentic AI system get fired from its job? 

Because it kept asking for more!

